In [2]:
import findspark
findspark.init("/usr/hdp/current/spark2-client/")

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder \
.appName("SparkSQL") \
.master("yarn") \
.getOrCreate()

In [6]:
online_retail_df = spark.read.format("csv") \
.option("header", True) \
.option("sep", ";") \
.option("inferSchema",True) \
.load("/user/erkan/OnlineRetail.csv")

In [7]:
online_retail_df.show(5)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2,55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2,75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 5 rows



In [8]:
online_retail_df.createOrReplaceTempView("tablo_adi")

In [9]:
spark.sql('''

SELECT COUNT(*) FROM tablo_adi



''').show(3)

+--------+
|count(1)|
+--------+
|  541909|
+--------+



In [10]:
spark.sql('''

SELECT Country, COUNT(*) AS SAYI FROM tablo_adi
GROUP BY Country
ORDER BY SAYI DESC



''').show(30)

+--------------------+------+
|             Country|  SAYI|
+--------------------+------+
|      United Kingdom|495478|
|             Germany|  9495|
|              France|  8557|
|                EIRE|  8196|
|               Spain|  2533|
|         Netherlands|  2371|
|             Belgium|  2069|
|         Switzerland|  2002|
|            Portugal|  1519|
|           Australia|  1259|
|              Norway|  1086|
|               Italy|   803|
|     Channel Islands|   758|
|             Finland|   695|
|              Cyprus|   622|
|              Sweden|   462|
|         Unspecified|   446|
|             Austria|   401|
|             Denmark|   389|
|               Japan|   358|
|              Poland|   341|
|              Israel|   297|
|                 USA|   291|
|           Hong Kong|   288|
|           Singapore|   229|
|             Iceland|   182|
|              Canada|   151|
|              Greece|   146|
|               Malta|   127|
|United Arab Emirates|    68|
+---------

In [12]:
spark.sql('''

SELECT Country, COUNT(*) AS SAYI FROM tablo_adi
GROUP BY Country
ORDER BY SAYI DESC



''').show(10)

+--------------+------+
|       Country|  SAYI|
+--------------+------+
|United Kingdom|495478|
|       Germany|  9495|
|        France|  8557|
|          EIRE|  8196|
|         Spain|  2533|
|   Netherlands|  2371|
|       Belgium|  2069|
|   Switzerland|  2002|
|      Portugal|  1519|
|     Australia|  1259|
+--------------+------+
only showing top 10 rows



In [11]:
from pyspark.sql import functions as f
online_retail_df.groupBy("Country") \
.agg(f.count("*").alias("SAYI")) \
.sort(f.desc("SAYI")) \
.show(10)

+--------------+------+
|       Country|  SAYI|
+--------------+------+
|United Kingdom|495478|
|       Germany|  9495|
|        France|  8557|
|          EIRE|  8196|
|         Spain|  2533|
|   Netherlands|  2371|
|       Belgium|  2069|
|   Switzerland|  2002|
|      Portugal|  1519|
|     Australia|  1259|
+--------------+------+
only showing top 10 rows



In [13]:
online_retail_df.groupBy("Country") \
.agg(f.count("*").alias("SAYI")) \
.sort(f.desc("SAYI")).explain()

== Physical Plan ==
*Sort [SAYI#117L DESC NULLS LAST], true, 0
+- Exchange rangepartitioning(SAYI#117L DESC NULLS LAST, 200)
   +- *HashAggregate(keys=[Country#19], functions=[count(1)])
      +- Exchange hashpartitioning(Country#19, 200)
         +- *HashAggregate(keys=[Country#19], functions=[partial_count(1)])
            +- *FileScan csv [Country#19] Batched: false, Format: CSV, Location: InMemoryFileIndex[hdfs://sandbox-hdp.hortonworks.com:8020/user/erkan/OnlineRetail.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Country:string>


In [14]:
spark.sql('''
SELECT Country, COUNT(*) AS SAYI FROM tablo_adi
GROUP BY Country
ORDER BY SAYI DESC
''').explain()

== Physical Plan ==
*Sort [SAYI#124L DESC NULLS LAST], true, 0
+- Exchange rangepartitioning(SAYI#124L DESC NULLS LAST, 200)
   +- *HashAggregate(keys=[Country#19], functions=[count(1)])
      +- Exchange hashpartitioning(Country#19, 200)
         +- *HashAggregate(keys=[Country#19], functions=[partial_count(1)])
            +- *FileScan csv [Country#19] Batched: false, Format: CSV, Location: InMemoryFileIndex[hdfs://sandbox-hdp.hortonworks.com:8020/user/erkan/OnlineRetail.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Country:string>


# Tutar bakımından En çok sipariş iptal etmiş müşteriler

In [17]:
online_retail_df.filter(online_retail_df.InvoiceNo.startswith('C')).show(4)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|  C536379|        D|            Discount|      -1|1.12.2010 09:41|     27,5|     14527|United Kingdom|
|  C536383|   35004C|SET OF 3 COLOURED...|      -1|1.12.2010 09:49|     4,65|     15311|United Kingdom|
|  C536391|    22556|PLASTERS IN TIN C...|     -12|1.12.2010 10:24|     1,65|     17548|United Kingdom|
|  C536391|    21984|PACK OF 12 PINK P...|     -24|1.12.2010 10:24|     0,29|     17548|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 4 rows



## TotalPrice yeni bir sütunda

In [22]:
from pyspark.sql import functions as f
online_retail_df2 = online_retail_df.withColumn("UnitPrice", 
                            f.regexp_replace(f.col("UnitPrice"), ',', '.').cast("double"))

online_retail_df2.show()

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1.12.2010 08:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|1.12.2010 08:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|1.12.2010 08:

In [28]:
online_retail_df3 = online_retail_df2.withColumn("TotalPrice", online_retail_df2.Quantity * online_retail_df2.UnitPrice)
online_retail_df3.show(3)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+------------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|        TotalPrice|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2.55|     17850|United Kingdom|15.299999999999999|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3.39|     17850|United Kingdom|             20.34|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2.75|     17850|United Kingdom|              22.0|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+------------------+
only showing top 3 rows



In [29]:
online_retail_df3.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- TotalPrice: double (nullable = true)



In [30]:
online_retail_df3.filter(" InvoiceNo LIKE 'C%' ") \
.groupBy("CustomerID") \
.agg(f.sum("TotalPrice").alias("SUM")) \
.sort("SUM") \
.show(5)

+----------+-------------------+
|CustomerID|                SUM|
+----------+-------------------+
|         0|          -285470.4|
|     16446|          -168469.6|
|     12346|           -77183.6|
|     15098|           -39267.0|
|     16029|-30032.230000000003|
+----------+-------------------+
only showing top 5 rows



In [31]:
online_retail_df3.createOrReplaceTempView("online_retail_df3")

In [35]:
spark.sql('''

SELECT CustomerID, SUM(TotalPrice) AS SUM FROM online_retail_df3
WHERE InvoiceNo LIKE "C%"
GROUP BY CustomerID
ORDER BY SUM

''').show(3)

+----------+---------+
|CustomerID|      SUM|
+----------+---------+
|         0|-285470.4|
|     16446|-168469.6|
|     12346| -77183.6|
+----------+---------+
only showing top 3 rows



## concat

In [39]:
online_retail_df3.withColumn("Invoice-StockCode", f.concat(f.col("InvoiceNo"), f.lit(" - "), f.col("StockCode"))) \
.limit(20).toPandas().head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,Invoice-StockCode
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,1.12.2010 08:26,2.55,17850,United Kingdom,15.30,536365 - 85123A
1,536365,71053,WHITE METAL LANTERN,6,1.12.2010 08:26,3.39,17850,United Kingdom,20.34,536365 - 71053
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,1.12.2010 08:26,2.75,17850,United Kingdom,22.00,536365 - 84406B
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,1.12.2010 08:26,3.39,17850,United Kingdom,20.34,536365 - 84029G
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,1.12.2010 08:26,3.39,17850,United Kingdom,20.34,536365 - 84029E


## format_number

In [40]:
online_retail_df3.withColumn("TotalPrice", f.format_number(f.col("TotalPrice"),4)) \
.limit(10).toPandas().head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,1.12.2010 08:26,2.55,17850,United Kingdom,15.3000
1,536365,71053,WHITE METAL LANTERN,6,1.12.2010 08:26,3.39,17850,United Kingdom,20.3400
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,1.12.2010 08:26,2.75,17850,United Kingdom,22.0000
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,1.12.2010 08:26,3.39,17850,United Kingdom,20.3400
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,1.12.2010 08:26,3.39,17850,United Kingdom,20.3400


# En çok tekrarlanan kelime wordcount

In [41]:
basit_metin = spark.read.format("text") \
.load("/user/erkan/basit_metin.txt")

In [42]:
basit_metin.show()

+--------------------+
|               value|
+--------------------+
|  Spark öğreniyorum.|
|Hem Python hem de...|
|Kurs sonunda hem ...|
+--------------------+



In [45]:
basit_metin_expl = basit_metin.select(f.explode(f.split(f.col("value"), "\s+")))
basit_metin_expl.show(3)

+------------+
|         col|
+------------+
|       Spark|
|öğreniyorum.|
|         Hem|
+------------+
only showing top 3 rows



In [47]:
basit_metin_expl.groupBy("col") \
.count() \
.sort(f.desc("count")) \
.show()

+-------------+-----+
|          col|count|
+-------------+-----+
|          hem|    3|
|        Spark|    3|
|       Python|    2|
|        Scala|    2|
|           de|    2|
|    olacağım.|    1|
|          çok|    1|
|         Kurs|    1|
|      sonunda|    1|
| öğreniyorum.|    1|
|yeteneklerimi|    1|
|    kullanmak|    1|
|      heyecan|    1|
|           ve|    1|
|          Hem|    1|
|      verici.|    1|
|          ile|    1|
|  geliştirmiş|    1|
+-------------+-----+

